## Retrieving Volcano Locations
___

We have access to a datafile with 65 different volcano locations and approximate aperture sizes. We will use this to extract location specific data fromt the [MODVOLC](http://modis.higp.hawaii.edu/cgi-bin/modisnew.cgi) database.

In [1]:
import pandas as pd
from os import mkdir
from os.path import isdir

In [2]:
df_loc = pd.read_csv("data/volcanoes.txt", delimiter='\s', header=None, engine='python');

df_loc.columns = ["name", "long", "lat", "aperture"]

df_loc.head()

,name,long,lat,aperture
0,ambrym,168.12,-16.25,0.2
1,anatahan,145.67,16.35,0.2
2,aoba,167.83,-15.40,0.2
3,arenal,-84.71,10.46,0.2
4,asama,138.53,36.40,0.2


In [3]:
## Here I am selecting Kilauea as a target
target = df_loc[df_loc.name == 'kilauea'].values.flatten()

data_dir = f"data/{target[0]}/"

if not isdir(data_dir):
    mkdir(data_dir)

## Retrieving the Data for the Target
___

We have selected Kilauea as a temporary test target. Now, we will draw a box around the region and extract the data from MODVOLC.

In [4]:
from get_data import get_data

In [5]:
jyear = 2019
jday = 100
jperiod = 100000
lonmin = (target[1] - 2*target[3])
lonmax = (target[1] + 2*target[3])
latmin = (target[2] - 2*target[3])
latmax = (target[2] + 2*target[3])

df = get_data(jyear=jyear, jday=jday, jperiod=jperiod, lonmin=lonmin, lonmax=lonmax, latmin=latmin, latmax=latmax).sort_values('UNIX_Time').reset_index()
df.rename(index=str, columns={"Mo": "Month", "Dy": "Day", "Hr": "Hour", "Mn" : "Minute"}, inplace=True)

df["datetime"] = pd.to_datetime(df[["Year", "Month", "Day", "Hour", "Minute"]])
print(f"...{df.shape[0]} measurements retrieved...")

df.to_csv(f"{data_dir}data.csv")
df.head()

...71861 measurements retrieved...


,index,UNIX_Time,Sat,Year,Month,Day,Hour,Minute,Longitude,Latitude,...,SunZen,SunAzi,Line,Samp,Ratio,Glint,Excess,Temp,Err,datetime
0,71860,951512100,T,2000,2,25,20,55,-155.367538,19.778879,...,37.76,137.23,327,205,-0.58,75.460,57.119,276.8,5.6,2000-02-25 20:55:00
1,71859,951512100,T,2000,2,25,20,55,-155.351715,19.777077,...,37.76,137.25,327,206,-0.41,75.392,83.206,276.8,5.6,2000-02-25 20:55:00
2,71858,951557099,T,2000,2,26,9,25,-155.053894,19.342176,...,161.06,-59.89,1145,69,-0.79,137.327,11.860,289.2,5.9,2000-02-26 09:25:00
3,71855,951600899,T,2000,2,26,21,35,-155.027740,19.336313,...,31.05,151.53,1532,1148,-0.56,32.510,51.439,289.2,5.9,2000-02-26 21:35:00
4,71857,951600899,T,2000,2,26,21,35,-155.096741,19.372599,...,31.11,151.45,1530,1144,-0.26,32.356,114.117,288.4,4.7,2000-02-26 21:35:00
